# Gamblers ruin

In this exercise we are going to look at an famous Markov chain problem known as Gamblers ruin.  Questions on this particular Markov chain appear on exam papers about Stochastic Processes that are set at many universities.  These questions will often have a structure something like the one below:

Suppose a gambler enters a casino with 4 pounds to his name.  The game he choses to play involves beting one pound and he can either win or loose.  If he wins he wins back double his stake so if he enters with 4 pounds and wins the first game he then has 5 pounds.  If by contrast he looses the game he looses his stake so if he enters with 4 pounds and looses the first game he now has only 3 pounds.  The gambler decides to stop playing if:

- He runs out of money.  In other words he stops playing once he has 0 pounds.
- Or if he has 10 pounds.

If the probability of winning each individual game is equal to 0.45 and if the outcomes of all the games are independent calculate the probability that the gambler looses all his money and the expected number of games he would play.

In this exercise we are going to write a program to simulate this gambler.  Before we get on to that first of all you need to take some time to understand the problem.  Hence, in your notes on this exercise:

- Explain why the amount of money the gambler has can be modelled using a Markov chain.
- Explain how many states there are in this Markov chain.
- Draw the transition graph for this Markov chain.
- Identify which states are transient and whcih states are recurrent in this Markov chain.
- Write out the transition matrix for this Markov chain.

Once you have completed these tasks then move on to the remainder of the exercises contained herein.  As always these tasks begin with some code for dynamic plotting that I have written for you so please press shift and enter on the cell below to begin.

In [ ]:
import math
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rc
from IPython.display import HTML
import random
rc('animation', html='none')

class plotobj(object) :
    def __init__(self,ngen):
        self.ngen = ngen
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(0,20), ylim=(0,10))
        
    def setup(self) :
        self.lines = []
        for i in range(0,self.ngen):
            lobj = self.ax.plot([],[],'-')[0]
            self.lines.append(lobj)
        
    def run(self,data):
        indata=data
        n,k=0,0
        xdata, ydata = [], []
        for i in range(0,self.ngen):
            xdata.append([])
            ydata.append([])
    
        maxx=0
        for p in indata :
            if ( p==0 ) | ( p==10 ) :
                xdata[n].append(k)
                if( k>maxx ) :
                    maxx = k
                ydata[n].append(p)
                self.lines[n].set_data( xdata[n], ydata[n] )
                k=0
                n += 1
            else :
                xdata[n].append(k)
                ydata[n].append(p)
                k += 1
        if( n<self.ngen ) :
            if( k>maxx ) :
                maxx = k
            self.lines[n].set_data( xdata[n], ydata[n] )
        self.ax.set_xlim(0,maxx)
        return self.lines,
        
def gamblersplot( ngen, operation ):
    myplot = plotobj( ngen )
    return anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=operation( ngen ), 
                                init_func=myplot.setup, interval=10, blit=False, repeat=False )

class plotobj3(object):
    def __init__(self,xmin,xmax):
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(xmin, xmax), ylim=(0, 1))
        
    def setup(self):
        self.line1, = self.ax.plot([],[],linestyle=':', marker='o', color='b')
        return self.line1
    
    def run(self,data):
        xdata,ydata = data
        self.ax.set_ylim(0,max(ydata))
        self.line1.set_data( xdata, ydata )
        return self.line1
    
class histoclass(object) :
    def __init__( self, xmin, xmax ):
        self.xmin=xmin
        self.xmax=xmax
        self.xdata = []
        self.ycount = []
        nbins = xmax - xmin + 2
        for i in range(0,nbins) :
            self.xdata.append( xmin+i )
            self.ycount.append(0)
            
    def buildHistogram( self, ngen, myvar, startstate ) :
        cnt=0
        while cnt<ngen :
            cnt += 1
            X = myvar(startstate)
            if( X >= self.xmax ) :
                continue
            nbin = int(X - self.xmin)
            self.ycount[nbin] += 1
            tdata = []
            for dat in self.ycount :
                tdata.append( dat / cnt )
            yield self.xdata, tdata
    
def histoplot( ngen, xmin, xmax, myvar, startstate ):
    myplot = plotobj3(  xmin, xmax )
    myhisto = histoclass( 0, 20 )
    return anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=myhisto.buildHistogram( ngen, myvar, startstate ), 
                                init_func=myplot.setup, interval=10, blit=False, repeat=False)

# Generating gambling instances

I think that it is useful to think about this problem in terms of a hierarchy of randomness.  The outcome of each individual game is random and as such the eventual outcome - either the gambler leaving with 10 pounds to his name or leaving with nothing - is random.  In addition, the number of games he will play before leaving the casino is also random.  When we look at things this way we realise that this complicated random object is ultimately constructed from two relatively-simple mathematical objects that we understand.  We just using a Bernoulli random variable and the number line (counting).

Another way of thinking about this problem is as a random walk with discrete steps along the number line.  To be clear there are 11 states in the chain (having 0, 1, 2, $\dots$, or 10 pounds).  Obviously, the order in which the gambler will visit these states is random.  Furthermore, if there are  two gamblers they will not visit these states in the same order.  To illustrate some possible outcomes I have written the following code.  If you press shift and enter on the cell below the set of states visited by 9 independent gamblers will be shown in a new window.  

In [ ]:
def gengambler( ngen ):
    npaths=0
    # This variable ( mypos ) is used to track the amount of money the gambler has
    # it is set equal to 4 initially as he starts with 4 pounds.
    mypos=4
    data = []
    data.append(4)
    # Here we use a while loop and we stop once the number of visits to 
    # either state 0 or state 10 are equal to ngen.  In your programs you
    # will also need to use a while loop.  When should your program stop?
    while npaths<ngen :
        # The probability of winning (and increasing the gamblers pot of money)
        # by one is 0.45.  We thus generate a uniform random variable between 0
        # and 1.  If it is less than 0.45 we increase mypos (the amount of money 
        # the gambler has by one).
        if( random.uniform(0,1)<0.45 ) :
            mypos += 1
        # If we don't win we loose and the gambler's pot of money decreases by one pound 
        else :
            mypos -= 1
        data.append(mypos)
        # A path ends whenever the gambler has no more pounds left to gamble or whenever
        # he has 10 pounds and choses to stop gambling.
        if (mypos==0) | (mypos==10) :
            npaths+=1
            if npaths<ngen :
                data.append(4)
            mypos=4
        yield data
        
gamblersplot( 10, gengambler )

It is worth noting a few things about these paths.  

- Different numbers of games are played by each of the gamblers before they arrive in either state 10 or state 0 and stop playing.
- Some gamblers finish with the 10 pounds and some loose all there money.

With this in mind we are going to investigate the following in the remainder of this exercise.

- We are going to try to estimate the average number of games a person plays in the casino.
- We are going to try to estimate the probabilty that gamblers will leave the casino with no money.

As in many of the other tasks we are going to estimate these quantities by generating a large number of random variables and estimating the expectation of a random variable by exploting the law of large numbers.  Without further ado then here is some code to dynamically plot expectation values.

In [ ]:
class plotobj2(object) :
    def __init__(self, ngen ) :
        self.t=0
        self.rsum=0
        self.ngen=ngen
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(0, self.ngen), ylim=(0, 1))
    
    def setup(self):
        self.xdata=[]
        self.ydata=[]
        self.line, = self.ax.plot([],[],'.')
        return self.line,
    
    def run(self,data):
        t,y = data
        self.xdata.append(t)
        self.ydata.append(y)
        self.line.set_data(self.xdata, self.ydata )
        return self.line,
           
def mean( ngen, myvar, startstate ):
    rsum = mean.rsum
    cnt = 0
    while cnt < ngen :
        cnt += 1
        rsum += myvar(startstate)
        yield cnt, rsum / cnt
        
def dynamicplot( ngen, myvar, startstate ):
    myplot = plotobj2( ngen )
    mean.rsum=0
    return anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=mean( ngen, myvar, startstate ), 
                                init_func=myplot.setup, interval=10, blit=False, repeat=False)

In the cell below write a code to generate a random variable that is equal to one when the gambler leaves the casino with no money and that is equal to zero when the gambler leaves the casino with 10 pounds and that takes in the amount of money he starts off with as an input parameter.  Notice that I have provided comments in the code I wrote for generating random paths in order to help you with this task.  Also note that I have provided a call to the dynamic plotting utitities above so that you can get a graph showing you how this expectation converges to a finite value.

In [ ]:
# You need to write the content of this function as at the moment the 
# function just returns 1, which is not very random.
def gamblerlooses( startstate ) :
    return 1

# The arguments here are
# Number of variables to generate = 1000
# function that generates the random variables = gambler looses
# Amount of money the gambler starts with = 4
dynamicplot( 1000,  gamblerlooses, 4 )

Now that we have examined the probability that gambler wins the game lets turn to the expected number of games he will play.  In the cell below write a code to generate a random variable that measures the number of games played before absorbtion.  This random number should take as an argument the state the amount of money the gambler starts with.  The comments in my program will again help you with this task.  The cell also contains a call to my  dynamic plotting tool so you can look at the convergence of this random variable.

In [ ]:
# Write a program that generates a random variable that measures the length of the path
# Notice this function takes the initial amount of money the gambler has as an input.
def genlength( startstate ) :
    return 1

# The arguments here are
# Number of variables to generate = 1000
# function that generates the random variables = gambler looses
# Amount of money the gambler starts with = 4
dynamicplot( 1000,  genlength, 4 )

In an earlier cell I have provided the tools that we used to calculate a histogram from a large number of discrete random variables.  The cell below allows you to use use this tool to plot the a histogram of values for the number of games played by the gambler prior to ruin.  In your notes explain the range of values can this random variable take? 

In [ ]:
# This will draw the histogram.  The arguments for this function are:
# The number of random samples to draw = 1000
# The minimum for the x axis = 0
# The maximum for the x axis = 50
# The function to use for generating random variables = genpath (function you have just written above)
# The amount of money the gambler starts with  = 4
histoplot( 1000, 0, 50, genlength, 4 )

The remainder of your notes on the gamblers ruin problem should focus on the analytical solution to this problem.  I will give you a separate handout that will take you through this process.